<h3> prøver å gjenskape henning sin 143.</h3>
<p><p>

In [1]:
%load_ext autoreload

In [2]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

/Users/miksx/.pyenv/versions/3.10.12/envs/TDT4173/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
%autoreload
from pipeline import Pipeline
pipin = Pipeline()

Current working directory: /Users/miksx/GitHub/Forest-Gump/mikael/autoML/gluon


In [4]:
PATH = "ag_144_exstract_time_4_16"

In [5]:
df1_0 = pipin.get_data("A")
df2_0 = pipin.get_data("B")
df3_0 = pipin.get_data("C")

In [6]:
numeric_columns = df1_0.select_dtypes(include='number').columns

# Select columns with negative values, excluding datetime columns
negative_columns = numeric_columns[(df1_0[numeric_columns] < 0).any()]
negative_columns

Index([], dtype='object')

In [7]:

df1_0 = pipin.remove_consecutive_measurments_new(df1_0, 4, 16)
df2_0 = pipin.remove_consecutive_measurments_new(df2_0, 4, 16)
df3_0 = pipin.remove_consecutive_measurments_new(df3_0, 4, 16)

/Users/miksx/GitHub/Forest-Gump/mikael/autoML/gluon/pipeline.py:250: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['consecutive_group'] = df.groupby(
/Users/miksx/GitHub/Forest-Gump/mikael/autoML/gluon/pipeline.py:253: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["is_first_in_consecutive_group"] = False
/Users/miksx/GitHub/Forest-Gump/mikael/autoML/gluon/pipeline.py:250: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consid

In [8]:
print(df1_0.shape, df2_0.shape, df3_0.shape)

(27348, 165) (19262, 165) (16311, 165)


In [9]:
df1_0.describe()

,"('absolute_humidity_2m:gm3', 0)","('absolute_humidity_2m:gm3', 15)","('absolute_humidity_2m:gm3', 30)","('absolute_humidity_2m:gm3', 45)","('air_density_2m:kgm3', 0)","('air_density_2m:kgm3', 15)","('air_density_2m:kgm3', 30)","('air_density_2m:kgm3', 45)","('ceiling_height_agl:m', 0)","('ceiling_height_agl:m', 15)",...,"('visibility:m', 45)","('wind_speed_10m:ms', 0)","('wind_speed_10m:ms', 15)","('wind_speed_10m:ms', 30)","('wind_speed_10m:ms', 45)",day_of_year_sin,hour,estimated,observed,pv_measurement
count,27340.000000,27340.000000,27340.000000,27340.000000,27340.000000,27340.000000,27340.000000,27340.000000,20748.000000,22133.000000,...,27340.000000,27340.000000,27340.000000,27340.000000,27340.000000,27340.000000,27340.000000,27340.000000,27340.000000,27348.000000
mean,6.846087,6.846316,6.846054,6.846364,1.246915,1.246908,1.246913,1.246911,2957.620117,3055.985596,...,32074.304688,3.010186,3.010201,3.010395,3.010684,0.690520,0.614367,0.100951,0.899049,785.937567
std,2.797055,2.794673,2.793597,2.794374,0.035344,0.035298,0.035276,0.035283,2618.337646,2653.875488,...,17609.863281,1.774386,1.758940,1.753278,1.758696,0.299733,0.316155,0.301269,0.301269,1253.837340
min,0.700000,0.700000,0.700000,0.700000,1.145000,1.145000,1.145000,1.145000,27.799999,27.799999,...,265.299988,0.000000,0.000000,0.000000,0.000000,0.004301,0.000000,0.000000,0.000000,0.000000
25%,4.600000,4.600000,4.600000,4.600000,1.223000,1.223000,1.223000,1.223000,1091.250000,1138.199951,...,15947.100586,1.600000,1.600000,1.600000,1.600000,0.468965,0.258819,0.000000,1.000000,0.000000
50%,6.700000,6.700000,6.700000,6.700000,1.245000,1.245000,1.245000,1.244000,1910.650024,1992.000000,...,35500.796875,2.700000,2.700000,2.700000,2.700000,0.797102,0.707107,0.000000,1.000000,119.350000
75%,8.800000,8.800000,8.800000,8.800000,1.270000,1.270000,1.270000,1.270000,4066.824951,4285.799805,...,47186.050781,4.100000,4.100000,4.100000,4.100000,0.952769,0.866025,0.000000,1.000000,1030.865000
max,17.500000,17.400000,17.400000,17.400000,1.427000,1.426000,1.426000,1.426000,12294.900391,12294.900391,...,70518.203125,11.400000,11.300000,11.100000,11.100000,0.999999,1.000000,1.000000,1.000000,5733.420000


In [10]:
train1 = TabularDataset(df1_0)
train2 = TabularDataset(df2_0)
train3 = TabularDataset(df3_0)

In [11]:
predictor1 = TabularPredictor(label="pv_measurement", eval_metric='mean_absolute_error',
                              path=PATH+"A").fit(
    train1,
    # time_limit=60,
    presets='best_quality',
    # tuning_data = tuning1,
    # use_bag_holdout=True,

    # verbosity = 3
    )

Presets specified: ['best_quality']
/Users/miksx/.pyenv/versions/3.10.12/envs/TDT4173/lib/python3.10/site-packages/autogluon/core/utils/utils.py:564: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context("mode.use_inf_as_na", True):  # treat None, NaN, INF, NINF as NA
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "ag_144_exstract_time_4_16A"
AutoGluon Version:  0.8.3b20231108
Python Version:     3.10.12
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 23.0.0: Fri Sep 15 14:41:43 PDT 2023; root:xnu-10002.1.13~1/RELEASE_ARM64_T6000
Disk Space Avail:   243.22 GB / 494.38 GB (49.2%)
Train Data Rows:    27348
Train Data Columns: 164
Label Column: pv_measurement
Preprocessing data ...
/Users/miksx/.pyenv/versions/3.10.12/envs/TD

In [12]:
test1 = pipin.get_test_data("A")
test_data1 = TabularDataset(test1)


pred1 = pd.DataFrame(predictor1.predict(test_data1))

/Users/miksx/.pyenv/versions/3.10.12/envs/TDT4173/lib/python3.10/site-packages/autogluon/tabular/models/fastainn/tabular_nn_fastai.py:200: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to downcast numeric results.
  df = df.fillna(column_fills, inplace=False, downcast=False)
/Users/miksx/.pyenv/versions/3.10.12/envs/TDT4173/lib/python3.10/site-packages/autogluon/tabular/models/fastainn/tabular_nn_fastai.py:200: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to downcast numeric results.
  df = df.fillna(column_fills, inplace=False, downcast=False)
/Users/miksx/.pyenv/versions/3.10.12/envs/TDT4173/lib/python3.10/site-packages/autogluon/tabular/models/fastainn/tabular_n

In [13]:
predictor2 = TabularPredictor(label="pv_measurement", eval_metric='mean_absolute_error',
                              path=PATH+"B").fit(
    train2,

    presets='best_quality', 
    # hyperparameters='very_large', 
    # time_limit=60,
    # tuning_data = tuning2,
    # use_bag_holdout=True,
    # verbosity = 3
    )

Presets specified: ['best_quality']
/Users/miksx/.pyenv/versions/3.10.12/envs/TDT4173/lib/python3.10/site-packages/autogluon/core/utils/utils.py:564: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context("mode.use_inf_as_na", True):  # treat None, NaN, INF, NINF as NA
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "ag_144_exstract_time_4_16B"
AutoGluon Version:  0.8.3b20231108
Python Version:     3.10.12
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 23.0.0: Fri Sep 15 14:41:43 PDT 2023; root:xnu-10002.1.13~1/RELEASE_ARM64_T6000
Disk Space Avail:   238.84 GB / 494.38 GB (48.3%)
Train Data Rows:    19262
Train Data Columns: 164
Label Column: pv_measurement
Preprocessing data ...
/Users/miksx/.pyenv/versions/3.10.12/envs/TD

In [14]:
predictor3 = TabularPredictor(label="pv_measurement", eval_metric='mean_absolute_error',
                              path=PATH+"C").fit(
    train3,

    presets='best_quality', 
    # hyperparameters='very_large', 
    # time_limit=60,
    # tuning_data = tuning3,
    # use_bag_holdout=True,
    # verbosity = 3
    )

Presets specified: ['best_quality']
/Users/miksx/.pyenv/versions/3.10.12/envs/TDT4173/lib/python3.10/site-packages/autogluon/core/utils/utils.py:564: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context("mode.use_inf_as_na", True):  # treat None, NaN, INF, NINF as NA
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "ag_144_exstract_time_4_16C"
AutoGluon Version:  0.8.3b20231108
Python Version:     3.10.12
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 23.0.0: Fri Sep 15 14:41:43 PDT 2023; root:xnu-10002.1.13~1/RELEASE_ARM64_T6000
Disk Space Avail:   230.61 GB / 494.38 GB (46.6%)
Train Data Rows:    16311
Train Data Columns: 164
Label Column: pv_measurement
Preprocessing data ...
/Users/miksx/.pyenv/versions/3.10.12/envs/TD

In [15]:
test1 = pipin.get_test_data("A")
test2 = pipin.get_test_data("B")
test3 = pipin.get_test_data("C")

In [16]:
test_data1 = TabularDataset(test1)
test_data2 = TabularDataset(test2)
test_data3 = TabularDataset(test3)

# pred1 = pd.DataFrame(predictor1.predict(test_data1))
pred2 = pd.DataFrame(predictor2.predict(test_data2))
pred3 = pd.DataFrame(predictor3.predict(test_data3))

negatives_pred1 = pred1[pred1["pv_measurement"] < 0]
negatives_pred2 = pred2[pred2["pv_measurement"] < 0]
negatives_pred3 = pred3[pred3["pv_measurement"] < 0]
neg = pd.concat([negatives_pred1, negatives_pred2, negatives_pred3])
neg.shape

/Users/miksx/.pyenv/versions/3.10.12/envs/TDT4173/lib/python3.10/site-packages/autogluon/tabular/models/fastainn/tabular_nn_fastai.py:200: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to downcast numeric results.
  df = df.fillna(column_fills, inplace=False, downcast=False)
/Users/miksx/.pyenv/versions/3.10.12/envs/TDT4173/lib/python3.10/site-packages/autogluon/tabular/models/fastainn/tabular_nn_fastai.py:200: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to downcast numeric results.
  df = df.fillna(column_fills, inplace=False, downcast=False)
/Users/miksx/.pyenv/versions/3.10.12/envs/TDT4173/lib/python3.10/site-packages/autogluon/tabular/models/fastainn/tabular_n

(75, 1)

In [17]:
pred = pd.concat([pred1, pred2, pred3])
final_prediction = pipin.post_processing(pred, prediction_column="pv_measurement")
final_prediction.to_csv('submissions/gluon_3_exstract_time_4_16_backwards.csv')

In [20]:
diff = pipin.compare_mae(final_prediction)
diff

21.53032577812647

In [19]:
# predictor.evaluate(df1, silent=True)